In [ ]:
import time

import tensorflow as tf
import numpy as np
from scipy import sparse as sp
from utils import *
from path_model import LRGCN
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix,roc_auc_score
from sklearn.metrics import precision_recall_fscore_support as score
import tf_slim as slim
from scipy import sparse
import pickle as pkl
import networkx as nx
from load_data import load_data
import sys
from collections import defaultdict

In [ ]:
tf.compat.v1.flags.DEFINE_integer('num_epochs', 25, 'number of epochs to train')
tf.compat.v1.flags.DEFINE_integer('batch_size', 1, 'batch size to train in one step')
tf.compat.v1.flags.DEFINE_integer('labels', 1, 'number of label classes')
tf.compat.v1.flags.DEFINE_integer('word_pad_length', 4438, 'word pad length for training')
tf.compat.v1.flags.DEFINE_integer('decay_step', 500, 'decay steps')
tf.compat.v1.flags.DEFINE_float('learn_rate', 1e-2, 'learn rate for training optimization')
tf.compat.v1.flags.DEFINE_boolean('train', False, 'train mode FLAG')
tf.compat.v1.flags.DEFINE_string('f','','')

FLAGS = tf.compat.v1.flags.FLAGS

In [ ]:
num_epochs = FLAGS.num_epochs
batch_size = FLAGS.batch_size
tag_size = FLAGS.labels
word_pad_length = FLAGS.word_pad_length
feature_dimension = 2
lr = FLAGS.learn_rate
window_size = 24
num_path = 200
max_path_len = 49

In [ ]:
tf.compat.v1.disable_eager_execution()
placeholders = {
    'support': [tf.compat.v1.sparse_placeholder(tf.float32) for _ in range(window_size)],
    'features':tf.compat.v1.placeholder(tf.float32, shape=(window_size,word_pad_length,feature_dimension)),
    'labels': tf.compat.v1.placeholder(tf.float32, shape=(num_path,)),
    'labels_mask': tf.compat.v1.placeholder(tf.int32,shape=(num_path,word_pad_length)),
    'dropout': tf.compat.v1.placeholder_with_default(0., shape=()),
    'path_node_index_array': tf.compat.v1.placeholder(tf.int32,shape=(num_path, max_path_len)),
    'num_features_nonzero': tf.compat.v1.placeholder(tf.int32)  # helper variable for sparse dropout
}

In [ ]:
model = LRGCN()

In [ ]:
with tf.compat.v1.Session() as sess:
  # build graph
  model.build_graph(n=word_pad_length,placeholders = placeholders,d =feature_dimension)
  # Downstream Application
  with tf.compat.v1.variable_scope('DownstreamApplication'):
    global_step = tf.Variable(0, trainable=False, name='global_step')
    learn_rate = tf.compat.v1.train.exponential_decay(lr, global_step, FLAGS.decay_step, 0.98, staircase=True)
    labels = placeholders['labels']
    Mask = placeholders['labels_mask']
    initializer = tf.keras.initializers.he_normal()
    print("---\n model.H2 shape: {}\n----".format(model.H2.shape))
    zero_padding = tf.constant(0.0, shape=[1, 8])
    rnn_input = tf.nn.embedding_lookup(tf.concat([model.H2, zero_padding], 0), placeholders['path_node_index_array'])
    sub_w11 = tf.compat.v1.get_variable(name='sub_w11',shape=(32,8),initializer=initializer)
    sub_w22 = tf.compat.v1.get_variable(name='sub_w22',shape=(8,32),initializer=initializer)
    lstm_cell = tf.compat.v1.nn.rnn_cell.LSTMCell(8)
    outputs,last_states=   tf.compat.v1.nn.dynamic_rnn(cell = lstm_cell,inputs = rnn_input, dtype = tf.float32)
    print("---\n outputs shape: {}\n----".format(outputs.shape))
    outputs_trans = tf.transpose(outputs, perm=[0, 2, 1])
    print("---\n outputs_trans shape: {}\n----".format(outputs_trans.shape))    
    sub_w11_stack = tf.tile(tf.expand_dims(sub_w11, 0), [num_path, 1, 1])
    sub_w22_stack = tf.tile(tf.expand_dims(sub_w22, 0), [num_path, 1, 1])
    attention_path = tf.nn.softmax(tf.matmul(sub_w22_stack, tf.tanh(tf.matmul(sub_w11_stack, outputs_trans))))
    print("---\n attention_path shape: {}\n----".format(attention_path.shape))

    path_output = tf.matmul(attention_path, outputs)
    print("---\n path_output shape: {}\n----".format(path_output.shape))
    path_output = tf.reshape(path_output, [num_path, 1, 8*8])
    print("---\n path_output shape: {}\n----".format(path_output.shape))
    initializer = tf.keras.initializers.he_normal()
    fc_weights = tf.compat.v1.get_variable(name='fc_weights',shape=(64,1),initializer=initializer)
    fc_weights_stack = tf.tile(tf.expand_dims(fc_weights, 0), [num_path, 1, 1])
    logits = tf.reshape(tf.matmul(path_output,fc_weights_stack),[-1])
    print("---\n logits shape: {}\n----".format(logits.shape))
    loss = tf.nn.weighted_cross_entropy_with_logits(targets=labels, logits=logits,pos_weight = 3.) 
    loss = tf.reduce_mean(loss)
    params = tf.compat.v1.trainable_variables()
    optimizer = tf.compat.v1.train.AdamOptimizer(learn_rate)
    grad_and_vars = tf.gradients(loss, params)
    clipped_gradients, _ = tf.clip_by_global_norm(grad_and_vars, 1)
    opt = optimizer.apply_gradients(zip(clipped_gradients, params), global_step=global_step)
    print("HERE3")


In [ ]:
def model_summary():
    model_vars = tf.trainable_variables()
    slim.model_analyzer.analyze_vars(model_vars, print_info=True)

In [ ]:
sess.run(tf.global_variables_initializer())